This notebook is used to generate dummy data for academic purposes.

In [15]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))
from modules import *

### Connected Solutions Enrollment Data

In [16]:
def create_dummy_cs_enrollment_tracker():
    program_tab_name_mapping = {
        "Cape Light Compact Program - Targeted Dispatch": "CLT",
        "Efficiency Maine-Demand Response Initiative": "EMT",
        "Eversource-Connected Solutions-Daily Dispatch": "EVD",
        "EVERSOURCE-Connected Solutions-Targeted Dispatch": "EVT",
        "Liberty-Connected Solutions-Targeted Dispatch": "LBT",
        "NGRID-Connected Solutions-Daily Dispatch": "NGD",
        "NGRID-Connected Solutions-Targeted Dispatch": "NGT",
        "Rhode Island Connected Solutions-Daily Dispatch": "RID",
        "Rhode Island Connected Solutions-Targeted Dispatch": "RIT",
        "Unitil-Connected Solutions-Daily Dispatch": "UND",
        "UNITIL-Connected Solutions-Targeted Dispatch": "UNT",
    }

    columns = [
        "ACCPRG", "Program", "Customer", "Customer Asset", "Asset ID", "Asset Type",
        "Utility Account Number", "Curtailment Strategy", "Building Type", "Customer Share",
        "Data Source", "Contracted KW", "Forecasted KW", "Meter Data Tags",
        "Performance Report Contact", "Statement Contact", "Third Party",
        "Third Party UOM", "Third Party Share"
    ]

    wb = Workbook()
    wb.remove(wb.active)

    for program_name, tab_name in program_tab_name_mapping.items():
        ws = wb.create_sheet(title=tab_name)
        
        num_rows = np.random.randint(10, 50)
        data = {
            "ACCPRG": [f"{tab_name}-{i:04d}" for i in range(num_rows)],
            "Program": [program_name for _ in range(num_rows)],
            "Customer": [f"Customer {i}" for i in range(num_rows)],
            "Customer Asset": [f"Asset {i}" for i in range(num_rows)],
            "Asset ID": [f"ID-{tab_name}-{i:04d}" for i in range(num_rows)],
            "Asset Type": np.random.choice(["HVAC", "Lighting", "Other"], num_rows),
            "Utility Account Number": [f"{np.random.randint(100000000, 999999999)}" for _ in range(num_rows)],
            "Curtailment Strategy": "Load Curtailment",
            "Building Type": np.random.choice(["Commercial", "Industrial", "Residential"], num_rows),
            "Customer Share": np.random.uniform(0.1, 0.9, num_rows).round(2),
            "Data Source": "Internal Database",
            "Contracted KW": np.random.uniform(50, 1000, num_rows).round(2),
            "Forecasted KW": np.random.uniform(40, 1200, num_rows).round(2),
            "Meter Data Tags": [f"Tag-{tab_name}-{i}" for i in range(num_rows)],
            "Performance Report Contact": [f"performance{i}@example.com" for i in range(num_rows)],
            "Statement Contact": [f"statement{i}@example.com" for i in range(num_rows)],
            "Third Party": np.random.choice(["Yes", "No"], num_rows),
            "Third Party UOM": np.random.choice(["kW", "kWh", "N/A"], num_rows),
            "Third Party Share": np.random.uniform(0.1, 0.5, num_rows).round(2)
        }

        df = pd.DataFrame(data, columns=columns)

        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(r)

    file_path = "../Enrollments/CS/Connected Solutions Enrollment Tracker.xlsx"
    wb.save(file_path)
    print(f"Dummy data has been saved to {file_path}")

create_dummy_cs_enrollment_tracker()

Dummy data has been saved to ../Enrollments/CS/Connected Solutions Enrollment Tracker.xlsx


### Dispatch Event Data

In [17]:
def generate_dispatch_events():
    enrollment_file = "../Enrollments/CS/Connected Solutions Enrollment Tracker.xlsx"
    enrollment_data = pd.read_excel(enrollment_file, sheet_name=None)
    all_enrollments = pd.concat(enrollment_data.values(), ignore_index=True)

    targeted_dates = ['2024-06-20', '2024-07-09', '2024-07-15', '2024-07-16', '2024-07-17']
    daily_dates = ['2024-06-20', '2024-07-01', '2024-07-02', '2024-07-03', '2024-07-05', 
                   '2024-07-06', '2024-07-07', '2024-07-08', '2024-07-09', '2024-07-10', 
                   '2024-07-11', '2024-07-12', '2024-07-13', '2024-07-14', '2024-07-15', 
                   '2024-07-16', '2024-07-17', '2024-07-18', '2024-07-19', '2024-07-21', 
                   '2024-07-22', '2024-07-23', '2024-07-24', '2024-07-25', '2024-07-26', 
                   '2024-07-28']

    all_events = []

    for _, enrollment in all_enrollments.iterrows():
        program = enrollment['Program']
        is_daily = 'Daily' in program
        event_dates = daily_dates if is_daily else targeted_dates

        for date in event_dates:
            start_time = datetime.strptime(f"{date} 14:00:00", "%Y-%m-%d %H:%M:%S")
            end_time = start_time + timedelta(hours=3)

            event = {
                'Region': 'ISONE',
                'Prgm Owner': program.split('-')[0],
                'Event Type': 'Mandatory',
                'Notification Type': 'Day Ahead',
                'Program': program,
                'Start Date & Time (Local)': start_time,
                'End Date & Time (Local)': end_time,
                'Company': enrollment['Customer'],
                'Facility': enrollment['Customer Asset'],
                'Accnt #': enrollment['Utility Account Number'],
                'Dispatch Target': f"{program.split('-')[0]} - Zone",
                'CRM #': enrollment['ACCPRG'],
                'Event Number': np.random.randint(3000, 4000),
                'AE': np.random.choice(['John Doe', 'Jane Smith', 'Bob Johnson']),
                'AM': np.random.choice(['Alice Brown', 'Charlie Davis', 'Eva White']),
                'AO': np.random.choice(['Frank Miller', 'Grace Lee', 'Henry Wilson'])
            }

            all_events.append(event)

    df_events = pd.DataFrame(all_events)
    output_file = "Dispatch Events.xlsx"
    df_events.to_excel(output_file, index=False)
    print(f"Dispatch events have been saved to {output_file}")

generate_dispatch_events()

Dispatch events have been saved to Dispatch Events.xlsx


### Dummy Load Data

In [18]:
def generate_dummy_load_data():
    enrollment_file = "../Enrollments/CS/Connected Solutions Enrollment Tracker.xlsx"
    enrollment_data = pd.read_excel(enrollment_file, sheet_name=None)
    all_enrollments = pd.concat(enrollment_data.values(), ignore_index=True)

    base_dir = "../Settlements/CS/Summer 2024/Raw Meter Data"
    os.makedirs(base_dir, exist_ok=True)

    start_date = datetime(2024, 6, 1)
    end_date = datetime(2024, 9, 30)
    date_range = pd.date_range(start=start_date, end=end_date, freq='15T')

    for _, enrollment in all_enrollments.iterrows():
        asset_id = enrollment['Asset ID']
        customer = enrollment['Customer']
        facility_name = enrollment['Customer Asset']
        program = enrollment['Program']
        contracted_kw = enrollment['Contracted KW']
        utility_account_number = enrollment['Utility Account Number']

        base_load = np.random.uniform(contracted_kw * 0.7, contracted_kw * 1.3, len(date_range))
        noise = np.random.normal(0, contracted_kw * 0.1, len(date_range))
        load_data = base_load + noise

        df = pd.DataFrame({
            'Time Intervals': date_range,
            'Load (kW)': load_data
        })

        if 'Daily' in program:
            event_type = "Daily Dispatch"
        else:
            event_type = "Targeted Dispatch"

        file_name = f"{event_type} Summer 2024 {customer} {facility_name} {asset_id} Raw Data.csv"
        file_path = os.path.join(base_dir, file_name)

        # Add metadata
        metadata = [
            f"Asset ID,{asset_id}",
            f"Utility Account Number,{utility_account_number}",
            f"Customer,{customer}",
            f"Date Range Collected,{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}",
            f"Date Retrieved,{datetime.now().strftime('%Y-%m-%d')}",
            f"Meter Tags,{enrollment['Meter Data Tags']}",
            ""
        ]

        with open(file_path, 'w') as f:
            f.write('\n'.join(metadata))
            df.to_csv(f, index=False)

    print("Dummy load data generation complete.")

generate_dummy_load_data()

C:\Users\Tyler.Sims\AppData\Local\Temp\ipykernel_26360\4146973443.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='15T')


Dummy load data generation complete.


#### Dummy Dynamics Data

In [19]:
def generate_dummy_dynamics_data():
    # Read the enrollment dummy file
    enrollment_file = "../Enrollments/CS/Connected Solutions Enrollment Tracker.xlsx"
    enrollment_data = pd.read_excel(enrollment_file, sheet_name=None)
    all_enrollments = pd.concat(enrollment_data.values(), ignore_index=True)

    # Create dummy data for utility and ISO accounts
    utility_data = []

    for _, enrollment in all_enrollments.iterrows():
        # Common data for both utility and ISO
        common_data = {
            "Enrollment Status": np.random.choice(["Enrolled", "Pending Enrollment", "Not Enrolled"], p=[0.8, 0.15, 0.05]),
            "Program": enrollment['Program'],
            "Name": enrollment['ACCPRG'],
            "Account": f"ACC-{np.random.randint(10000, 99999)}",
            "Vendor ID (Account) (Account)": f"VID-{np.random.randint(1000, 9999)}",
            "Company": enrollment['Customer'],
            "Facility Name/Store # (Account) (Account)": enrollment['Customer Asset'],
            "Address 1 (Company) (Company)": f"{np.random.randint(100, 9999)} Main St",
            "Service Address Line 1 (Account) (Account)": f"{np.random.randint(100, 9999)} Energy Ave",
            "Service Address Line 2 (Account) (Account)": f"Suite {np.random.randint(100, 999)}",
            "Service Address City (Account) (Account)": np.random.choice(["Boston", "Worcester", "Springfield", "Cambridge", "Lowell"]),
            "Service Address State (Account) (Account)": "MA",
            "Service Address Zip 1 (Account) (Account)": f"{np.random.randint(10000, 99999)}",
            "Service Address Latitude (Account) (Account)": np.random.uniform(41.0, 43.0),
            "Service Address Longitude (Account) (Account)": np.random.uniform(-73.0, -69.0),
            "Earliest Start Date": (datetime.now() - timedelta(days=np.random.randint(30, 365))).strftime('%Y-%m-%d'),
            "End Date": (datetime.now() + timedelta(days=np.random.randint(30, 730))).strftime('%Y-%m-%d'),
            "Utility (Account) (Account)": enrollment['Program'].split('-')[0],
            "Utility Account Number (Account) (Account)": enrollment['Utility Account Number'],
            "Asset ID": enrollment['Asset ID'],
            "Secondary ID (Account) (Account)": f"SID-{np.random.randint(1000, 9999)}",
            "Aggregation ID": f"AGG-{np.random.randint(100, 999)}",
            "Resource ID": f"RID-{np.random.randint(1000, 9999)}",
            "Resource Name": f"Resource-{enrollment['Customer Asset'][:10]}"
        }

        # Add to utility data
        utility_data.append(common_data)

    # Convert to DataFrames
    df_utility = pd.DataFrame(utility_data)

    # Save to Excel files
    df_utility.to_excel("Dynamics Source Data - Utility Accounts.xlsx", index=False)

    print("Dummy Dynamics data generation complete.")

# Run the function
generate_dummy_dynamics_data()

Dummy Dynamics data generation complete.
